In [9]:
# Importing libraries 

import pandas as pd
import numpy as np




In [ ]:
# Task 1: Data Loading and Initial Exploration

df = pd.read_csv("/Users/clairestewart/ML-fundamentals-2025/bike+sharing+dataset/hour.csv")

